In [9]:
import pandas as pd
import requests as rq
import os
import numpy as np
from bs4 import BeautifulSoup

O projeto usará como base os seguintes sites:
MLSSoccer - https://www.mlssoccer.com
Eurosport - https://www.eurosport.com
Soccerway - https://us.soccerway.com
FCTable - https://www.fctables.com
WhoScored - https://www.whoscored.com
FIFA - https://www.fifa.com
TopDrawerSoccer - https://www.topdrawersoccer.com
FoxSports - https://www.foxsports.com

In [24]:
sites = [('MLSSoccer','https://www.mlssoccer.com'),
('Eurosport','https://www.eurosport.com'),
('Soccerway','https://us.soccerway.com'),
('FCTable','https://www.fctables.com'),
('WhoScored','https://www.whoscored.com'),
('FIFA','https://www.fifa.com'),
('TopDrawerSoccer','https://www.topdrawersoccer.com'),
('FoxSports','https://www.foxsports.com')]
##Duvidas:
#Eurosport vai ter como root https://www.eurosport.com/ ou https://www.eurosport.com/football ?
#Foxsports vai ter como root https://www.foxsports.com/ ou https://www.foxsports.com/soccer ?

In [25]:
#Robots.txt de cada site
#for i in range(len(sites)):
# urlRobots = sites[i][1] + '/robots.txt'
# headers = {'User-Agent': 'bvcl'}
# req = rq.get(urlRobots, headers=headers)
# directoryFile = 'robots/' + sites[i][0] + ".txt"
# with open(directoryFile, 'wb') as handle:
#    for block in req.iter_content():
#        handle.write(block)

###Problema no site whoscored

In [12]:
#Pagina inicial de cada site
#for i in range(len(sites)):
#    urlHomePage = sites[i][1]
#    req = rq.get(urlHomePage)
#    directoryFile = 'pages/' + sites[i][0] + "/homePage.html" 
#    os.makedirs(os.path.dirname(directoryFile), exist_ok=True)
#    with open(directoryFile, 'wb') as handle:
#        for block in req.iter_content():
#            handle.write(block)

In [30]:
###Problema no whoscored
urlHomePage = sites[0][1]
req = rq.get(urlHomePage)
parsedPage = BeautifulSoup(req.text,"html.parser")
anchors = parsedPage.find_all('a', href = True)
#anchors

In [40]:
#Construindo array de allow e disallow do robots.txt
###Duvida: Levar em consideração Visit-time e Crawl-delay do robots ?
pageCounter = 0
robotsTXT = 'robots/' + sites[pageCounter][0] + ".txt"
with open(robotsTXT) as f:
   content = f.readlines()

start = len(content)
end = len(content)
disallowArray = []
allowArray = []
visitTime = ''
crawlDelay = ''
for i in range(len(content)):
   if('User-agent: *' in content[i]):
       start = i
       break
for i in range(start+1, len(content)):
   if('User-agent: ' in content[i]):
       end = i
       break
for i in range(start+1, len(content)):
   if('Visit-time: ' in content[i]):
       visitTime = content[i].split(' ')[1]
       break
for i in range(start+1, len(content)):
   if('Crawl-delay: ' in content[i]):
       crawlDelay = content[i].split(' ')[1].split('\n')[0]
       break
for i in range (start+1, end-1):
   line = content[i].split(' ')
   if(line[0] == 'Allow:'):
       allowArray.append(line[1].split('\n')[0])
   elif(line[0] == 'Disallow:'):
       disallowArray.append(line[1].split('\n')[0])
#disallowArray